In [ ]:
import json
import torch
import random
import numpy as np
from pathlib import Path
from audiotoken import AudioToken, Tokenizers
from IPython.display import Audio, display

In [ ]:
aco_encoder = AudioToken(tokenizer = Tokenizers.acoustic, device = 'cuda:1')

In [ ]:
root_dir = '/home/meraki/.cache/indri/youtube_en_spongebob_raw/'
data = []
with open(Path(root_dir, 'annotation/metadata.jsonl')) as fp:
    for ln in fp.readlines():
        if not ln:
            continue
        data.append(json.loads(ln))

print(len(data))

In [ ]:
k = random.choice(range(len(data)))
# k = 167156

actual_audio = Path(root_dir, data[k]['audio_path'])
# actual_audio = np.load(actual_audio)
print(k, len(data[k]['raw_text']), data[k]['raw_text'])
display(Audio(actual_audio, rate=16000))

reconstructed_audio = Path(root_dir, data[k]['acoustic_tokens'])
toks = np.load(reconstructed_audio)[:2]
reconstructed_audio = aco_encoder.decode(torch.from_numpy(toks).unsqueeze(0))
display(Audio(reconstructed_audio, rate=24000))

In [ ]:
toks.shape

In [ ]:
data[k]

In [ ]:
toks.shape

In [ ]:
14202

In [ ]:
167156, 45024

In [ ]:
import torch
import torchaudio
from encodec import EncodecModel
from encodec.utils import convert_audio

In [ ]:
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(1.5)

In [ ]:
from contextlib import nullcontext
ctx = nullcontext()
if torch.cuda.is_available():
    ctx = torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16) # type: ignore

In [ ]:
actual_audio = Path('/home/meraki/Downloads/output.wav')

In [ ]:
wav, sr = torchaudio.load(actual_audio)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.unsqueeze(0)
print(sr, wav.shape)

# with ctx:
with torch.no_grad():
    encoded_frames = model.encode(wav)

codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # [B, n_q, T]
print(codes.shape)

with torch.no_grad():
    reconstructed_wav = model.decode(encoded_frames)

display(Audio(wav[0], rate=24000))
display(Audio(reconstructed_wav[0], rate=24000))

In [ ]:
import bark
import numpy as np

In [ ]:
def _deserialize_acoustic_tokens(tokens):
    cb1 = tokens[::2][0]
    cb2 = tokens[1::2][0]
    acoustic_tokens = np.stack([cb1, cb2])

    return acoustic_tokens

_ = bark.generation.load_model(
   use_gpu=True,
   model_type="fine"
)

In [ ]:
def nar_bark(tokens_02):
    tokens_02 = _deserialize_acoustic_tokens(tokens_02)

    print(type(tokens_02), tokens_02.shape)
    with torch.no_grad():
        tokens = bark.api.generate_fine(
            x_coarse_gen=tokens_02[0:2, :],
            silent=False
        )
    tokens = np.expand_dims(tokens, axis=0)
    tokens = torch.from_numpy(tokens)

    return tokens

In [ ]:
codes[0].shape

In [ ]:
bark_gen = nar_bark(codes[0])

In [ ]:
type(bark_gen)

In [ ]:
aco_encoder.num_codebooks

In [ ]:
bark_aud = aco_encoder.decode(bark_gen)
display(Audio(bark_aud, rate=24000))

In [ ]:
bark_aud = aco_encoder.decode(bark_gen)
display(Audio(bark_aud, rate=24000))

In [ ]:
encoded_frames

In [ ]:
wav, sr = torchaudio.load(actual_audio)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.unsqueeze(0)

with 
with torch.no_grad():
    encoded_frames = model.encode(wav)

codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # [B, n_q, T]

with torch.no_grad():
    # out = model.quantizer.decode(codes)#.transpose(0, 1)
    # reconstructed_wav = model.decoder(out)
    reconstructed_wav = model.decode([(torch.from_numpy(toks).unsqueeze(0).to(torch.long), None)])

display(Audio(reconstructed_wav[0], rate=24000))